In [1]:
import seaborn
import pandas as pd
import nannyml as nml

I will use ana_data and ref_data.
<div style="background-color: #ffebee; border-left: 4px solid #f44336; padding: 15px; margin: 10px 0;">EDIT

In [2]:
# load the red_data dataset
reference= pd.read_csv('../data/03_primary/preprocessed_train_data_trees.csv')
analysis = pd.read_csv('../data/03_primary/preprocessed_test_data_trees.csv')

data = pd.concat([reference, analysis], ignore_index=True)
print(f"Combined data shape: {data.shape}")
print(f"Combined data columns: {data.columns}")
print(f"Reference data shape: {reference.shape}")
print(f"Analysis data shape: {analysis.shape}")

Combined data shape: (34168, 22)
Combined data columns: Index(['index', 'number_of_adults', 'number_of_children',
       'number_of_weekend_nights', 'number_of_week_nights', 'lead_time',
       'average_price', 'special_requests', 'date_of_reservation',
       'booking_status', 'market_segment_type_Corporate',
       'market_segment_type_Offline', 'market_segment_type_Online',
       'market_segment_type_Other', 'room_type_Other', 'room_type_Room_Type 1',
       'room_type_Room_Type 4', 'season_high', 'season_low',
       'type_of_meal_Meal Plan 1', 'type_of_meal_Meal Plan 2',
       'type_of_meal_Not Selected'],
      dtype='object')
Reference data shape: (25225, 22)
Analysis data shape: (8943, 22)


In [3]:
data.head()

,index,number_of_adults,number_of_children,number_of_weekend_nights,number_of_week_nights,lead_time,average_price,special_requests,date_of_reservation,booking_status,...,market_segment_type_Online,market_segment_type_Other,room_type_Other,room_type_Room_Type 1,room_type_Room_Type 4,season_high,season_low,type_of_meal_Meal Plan 1,type_of_meal_Meal Plan 2,type_of_meal_Not Selected
0,0,1,1,2,5,224,88.00,0,2015-10-02,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1,2,2,1,1,3,1,50.00,0,2018-02-28,1,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
2,3,1,0,0,2,211,100.00,1,2017-05-20,1,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,4,1,0,1,2,48,77.00,0,2018-04-11,1,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,6,1,1,1,4,34,107.55,1,2017-10-15,0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


I will define features to use for data drift based on Top 10 features by Permutation Importance: 

    - lead time
    - special_requests
    - average_price
    - market_segment_type_Offline
    - market_segment_type_Online
    - room_type_Room_Type 4
    - number_of_children
    - room_type_Room_Type 1
    - number_of_week_nights
    - type_of_meal_Meal Plan 2

<div style="background-color: #ffebee; border-left: 4px solid #f44336; padding: 15px; margin: 10px 0;">Will probably need to change this 

In [4]:
# Define feature columns
feature_column_names = ['lead_time',
    'special_requests',
    'average_price',
    'market_segment_type_Offline',
    'market_segment_type_Online',
    'room_type_Room_Type 4',
    'number_of_children',
    'room_type_Room_Type 1',
    'number_of_week_nights',
    'type_of_meal_Meal Plan 2']

feature_column_names

['lead_time',
 'special_requests',
 'average_price',
 'market_segment_type_Offline',
 'market_segment_type_Online',
 'room_type_Room_Type 4',
 'number_of_children',
 'room_type_Room_Type 1',
 'number_of_week_nights',
 'type_of_meal_Meal Plan 2']

In [5]:
data[feature_column_names].dtypes

lead_time                        int64
special_requests                 int64
average_price                  float64
market_segment_type_Offline    float64
market_segment_type_Online     float64
room_type_Room_Type 4          float64
number_of_children               int64
room_type_Room_Type 1          float64
number_of_week_nights            int64
type_of_meal_Meal Plan 2       float64
dtype: object

# Univariate drift detection

In [6]:
constant_threshold = nml.thresholds.ConstantThreshold(lower=0.1, upper=0.5)

## Categorical

In [7]:
# Let's initialize the object that will perform the Univariate Drift calculations
univariate_calculator = nml.UnivariateDriftCalculator(
    column_names=["market_segment_type_Offline", "market_segment_type_Online", "room_type_Room_Type 4", "room_type_Room_Type 1", "type_of_meal_Meal Plan 2"],
    treat_as_categorical=['market_segment_type_Offline', 'market_segment_type_Online', 'room_type_Room_Type 4', 'room_type_Room_Type 1', 'type_of_meal_Meal Plan 2'],
    chunk_size=100,
    timestamp_column_name='date_of_reservation',
    categorical_methods=['jensen_shannon'],
    thresholds={"jensen_shannon":constant_threshold},
)

In [9]:
univariate_calculator.fit(reference)
results = univariate_calculator.calculate(analysis)
display(results.filter(period='analysis', column_names=["market_segment_type_Offline", "market_segment_type_Online", "room_type_Room_Type 4", "room_type_Room_Type 1", "type_of_meal_Meal Plan 2"]).to_df())

chunk                                                          \
          chunk                                                           
            key chunk_index start_index end_index start_date   end_date   
0        [0:99]           0           0        99 2018-09-20 2018-09-21   
1     [100:199]           1         100       199 2018-09-21 2018-09-22   
2     [200:299]           2         200       299 2018-09-22 2018-09-22   
3     [300:399]           3         300       399 2018-09-22 2018-09-24   
4     [400:499]           4         400       499 2018-09-24 2018-09-25   
..          ...         ...         ...       ...        ...        ...   
85  [8500:8599]          85        8500      8599 2018-12-27 2018-12-27   
86  [8600:8699]          86        8600      8699 2018-12-27 2018-12-28   
87  [8700:8799]          87        8700      8799 2018-12-28 2018-12-29   
88  [8800:8899]          88        8800      8899 2018-12-29 2018-12-30   
89  [8900:8942]          89        8900      8942 2018-12-30 2018-12-31   

             market_segment_type_Offline                                  ...  \
                          jensen_shannon                                  ...   
      period                       value upper_threshold lower_threshold  ...   
0   analysis                    0.025966             0.5             0.1  ...   
1   analysis                    0.193101             0.5             0.1  ...   
2   analysis                    0.047610             0.5             0.1  ...   
3   analysis                    0.214104             0.5             0.1  ...   
4   analysis                    0.200805             0.5             0.1  ...   
..       ...                         ...             ...             ...  ...   
85  analysis                    0.251924             0.5             0.1  ...   
86  analysis                    0.200805             0.5             0.1  ...   
87  analysis                    0.065175             0.5             0.1  ...   
88  analysis                    0.045319             0.5             0.1  ...   
89  analysis                    0.039644             0.5             0.1  ...   

   room_type_Room_Type 4        room_type_Room_Type 1                  \
          jensen_shannon               jensen_shannon                   
         lower_threshold  alert                 value upper_threshold   
0                    0.1  False              0.199047             0.5   
1                    0.1   True              0.017671             0.5   
2                    0.1  False              0.121276             0.5   
3                    0.1  False              0.173565             0.5   
4                    0.1   True              0.094228             0.5   
..                   ...    ...                   ...             ...   
85                   0.1   True              0.017671             0.5   
86                   0.1   True              0.121276             0.5   
87                   0.1   True              0.094228             0.5   
88                   0.1  False              0.173565             0.5   
89                   0.1  False              0.129114             0.5   

                          type_of_meal_Meal Plan 2                  \
                                    jensen_shannon                   
   lower_threshold  alert                    value upper_threshold   
0              0.1  False                 0.023698             0.5   
1              0.1   True                 0.153405             0.5   
2              0.1  False                 0.158134             0.5   
3              0.1  False                 0.100644             0.5   
4              0.1   True                 0.126532             0.5   
..             ...    ...                      ...             ...   
85             0.1   True                 0.126532             0.5   
86             0.1  False                 0.158134             0.5   
87             0.1   True      

In [10]:
figure = results.filter(column_names=['market_segment_type_Offline', 'market_segment_type_Online', 'room_type_Room_Type 4', 'room_type_Room_Type 1', 'type_of_meal_Meal Plan 2'], period='analysis', methods=['jensen_shannon']).plot(kind='drift')
figure.show()# Plot drift results for all categorical columns

## Continuous features

In [11]:
# Let's initialize the object that will perform the Univariate Drift calculations
univariate_calculator = nml.UnivariateDriftCalculator(
    column_names=['average_price'],
    chunk_size=2500,
    timestamp_column_name='date_of_reservation',
    categorical_methods=['jensen_shannon'],
    thresholds={"jensen_shannon":constant_threshold},
)

In [12]:
univariate_calculator.fit(reference)
results = univariate_calculator.calculate(analysis)
display(results.filter(period='analysis', column_names=['average_price']).to_df())

/Users/zofiawojcik/MLOps/project_mlops/.venv/lib/python3.11/site-packages/nannyml/chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.



chunk                                                          \
         chunk                                                           
           key chunk_index start_index end_index start_date   end_date   
0     [0:2499]           0           0      2499 2018-09-20 2018-10-13   
1  [2500:4999]           1        2500      4999 2018-10-13 2018-11-04   
2  [5000:7499]           2        5000      7499 2018-11-04 2018-12-08   
3  [7500:8942]           3        7500      8942 2018-12-08 2018-12-31   

             average_price                                         
            jensen_shannon                                         
     period          value upper_threshold lower_threshold  alert  
0  analysis       0.307659             0.5             0.1  False  
1  analysis       0.235965             0.5             0.1  False  
2  analysis       0.254629             0.5             0.1  False  
3  analysis       0.216334             0.5             0.1  False

In [13]:
figure = results.filter(column_names=['average_price'], methods=['jensen_shannon']).plot(kind='distribution')
figure.show()

/Users/zofiawojcik/MLOps/project_mlops/.venv/lib/python3.11/site-packages/nannyml/chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.

/Users/zofiawojcik/MLOps/project_mlops/.venv/lib/python3.11/site-packages/plotly/io/_json.py:560: UserWarning:

Discarding nonzero nanoseconds in conversion.



## Chi-squared drift detection for categorical data

In [14]:
# Let's initialize the object that will perform the Univariate Drift calculations
univariate_calculator = nml.UnivariateDriftCalculator(
    column_names=['lead_time',
    'special_requests',
    'average_price',
    'market_segment_type_Offline',
    'market_segment_type_Online',
    'room_type_Room_Type 4',
    'number_of_children',
    'room_type_Room_Type 1',
    'number_of_week_nights',
    'type_of_meal_Meal Plan 2'],
    treat_as_categorical=['lead_time',
    'special_requests',
    'average_price',
    'market_segment_type_Offline',
    'market_segment_type_Online',
    'room_type_Room_Type 4',
    'number_of_children',
    'room_type_Room_Type 1',
    'number_of_week_nights',
    'type_of_meal_Meal Plan 2'],
    chunk_size=2500,
    timestamp_column_name='date_of_reservation',
    categorical_methods=['chi2']
)

In [15]:
univariate_calculator.fit(reference)
results = univariate_calculator.calculate(analysis)
display(results.filter(period='analysis', column_names=['lead_time',
    'special_requests',
    'average_price',
    'market_segment_type_Offline',
    'market_segment_type_Online',
    'room_type_Room_Type 4',
    'number_of_children',
    'room_type_Room_Type 1',
    'number_of_week_nights',
    'type_of_meal_Meal Plan 2']).to_df())

/Users/zofiawojcik/MLOps/project_mlops/.venv/lib/python3.11/site-packages/nannyml/chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.



chunk                                                          \
         chunk                                                           
           key chunk_index start_index end_index start_date   end_date   
0     [0:2499]           0           0      2499 2018-09-20 2018-10-13   
1  [2500:4999]           1        2500      4999 2018-10-13 2018-11-04   
2  [5000:7499]           2        5000      7499 2018-11-04 2018-12-08   
3  [7500:8942]           3        7500      8942 2018-12-08 2018-12-31   

               lead_time                                  ...  \
                    chi2                                  ...   
     period        value upper_threshold lower_threshold  ...   
0  analysis  6361.120974            None            None  ...   
1  analysis  6629.065942            None            None  ...   
2  analysis  2768.897908            None            None  ...   
3  analysis  3727.007941            None            None  ...   

  room_type_Room_Type 1        number_of_week_nights                  \
                   chi2                         chi2                   
        lower_threshold  alert                 value upper_threshold   
0                  None   True            121.373763            None   
1                  None  False            228.796142            None   
2                  None   True             40.493383            None   
3                  None   True            145.032962            None   

                        type_of_meal_Meal Plan 2                  \
                                            chi2                   
  lower_threshold alert                    value upper_threshold   
0            None  True                14.731943            None   
1            None  True                 4.420426            None   
2            None  True               104.358053            None   
3            None  True                15.694562            None   

                         
                         
  lower_threshold alert  
0            None  True  
1            None  True  
2            None  True  
3            None  True  

[4 rows x 47 columns]

In [16]:
figure = results.filter(column_names=['lead_time',
    'special_requests',
    'average_price',
    'market_segment_type_Offline',
    'market_segment_type_Online',
    'room_type_Room_Type 4',
    'number_of_children',
    'room_type_Room_Type 1',
    'number_of_week_nights',
    'type_of_meal_Meal Plan 2'],
    methods=['chi2']).plot(kind='distribution')
figure.show()

/Users/zofiawojcik/MLOps/project_mlops/.venv/lib/python3.11/site-packages/nannyml/chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.

/Users/zofiawojcik/MLOps/project_mlops/.venv/lib/python3.11/site-packages/nannyml/chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.

/Users/zofiawojcik/MLOps/project_mlops/.venv/lib/python3.11/site-packages/nannyml/chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.

/Users/zofiawojcik/MLOps/project_mlops/.venv/lib/python3.11/site-packages/nannyml/chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.

/Users/zofiawojcik/MLOps/project_mlops/.venv

In [5]:
return model, X_train.columns, results_dict, plt

SyntaxError: 'return' outside function (1170086711.py, line 1)